# Agent


![Screenshot 2024-08-21 at 12.44.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0ba0bd34b541c448cc_agent1.png)



![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

In [1]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def divide(a: int, b: int) -> float:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

llm = ChatOllama(model='llama3-groq-tool-use',temperature=0)
llm_with_tools = llm.bind_tools([multiply,add,divide])
call = llm_with_tools.invoke([HumanMessage(content="tell me all the tools needed for add 5 by 3, then multiply the result by 2 and finally divid it by 4")])
call.pretty_print()

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm_with_tools,tools=[multiply,add,divide])
calls = agent.invoke({"messages": "add 4 and 5 then multiply by 2 and divide by 3"})
calls

In [67]:
from typing import Any
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs. use all the tools")

class Agent: 
   def __init__(self, llm_with_tools) -> None:
      self.llm = llm_with_tools
   def __call__(self, state:MessagesState, config) :
      messages = state['messages']
      result = self.llm.invoke([sys_msg] + messages)
      return {"messages": result}

In [ ]:
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

builder = StateGraph(MessagesState)
builder.add_node("agent",Agent(llm_with_tools))
builder.add_node("tools", ToolNode([divide,add,multiply]))

builder.add_edge(START,'agent')
builder.add_conditional_edges('agent',tools_condition)
builder.add_edge('tools','agent')

workflow = builder.compile()
display(Image(workflow.get_graph().draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="add 5 by 3, then multiply the result by 2 and finally divid it by 4")]
messages = workflow.invoke({"messages": messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

## LangSmith

We can look at traces in LangSmith.

In [32]:
import os 
os.environ["LANGCHAIN_PROJECT"] = "one"

In [ ]:
!pip install langchain_groq


In [5]:
from langchain_groq.chat_models import ChatGroq
import os
os.environ["GROQ_API_KEY"] 

KeyError: 'GROQ_API_KEY'

In [ ]:

model = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", temperature=0)